In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 쿨루프 시공 대상 여부 분류
- Tasks
    1. Data Preprocessing
        - 모델 사용을 위해 데이터를 일관성 있게 정리해야 합니다.
    2. Object Detection
        - 전처리 된 데이터를 이용하여 학습을 진행하세요.
        - 인공위성 지도를 이용하여 이미지를 **추가적으로** 10장 수집하고 추론 과정에서 사용하세요.

### 데이터셋 다운로드 및 압축 해제
- cool_roof_image.zip : 이미지 데이터 압축 파일
- cool_roof_yolo_labels.zip : YOLO 모델 형식에 맞게 가공된 레이블 파일

In [ ]:
import gdown, zipfile
import os, glob, shutil
from tqdm import tqdm
import random
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2


In [ ]:
image_path = '/content/drive/MyDrive/KT_aivle/CNN/mini_4/cool_roof_images.zip'
label_path = "/content/drive/MyDrive/KT_aivle/CNN/mini_4/cool_roof_yolo_labels.zip"
test_path = "/content/drive/MyDrive/KT_aivle/CNN/mini_4/cool_roof_yolo_test.zip"
ex_train_path = '/content/drive/MyDrive/KT_aivle/CNN/mini_4/ex_train.zip'

In [ ]:
def dataset_extract(file_name) :
    with zipfile.ZipFile(file_name, 'r') as zip_ref :
        file_list = zip_ref.namelist()

        if os.path.exists(f'./{file_name[:-4]}/') :
            print(f'데이터셋 폴더가 이미 존재합니다.')
            return

        else :
            for f in tqdm(file_list, desc='Extracting', unit='files') :
                zip_ref.extract(member=f, path=f'./{file_name[-10:-4]}/')

In [ ]:
dataset_extract(image_path)

In [ ]:


dataset_extract(label_path)

In [ ]:
dataset_extract(test_path)

In [ ]:
dataset_extract(ex_train_path)

### 폴더 생성
- YOLO 모델에서 요구하는 폴더의 형식이 있습니다.
- 해당 형식에 맞춰 폴더를 만드세요.

In [ ]:
!mkdir /content/datasets
!mkdir /content/datasets/train; mkdir /content/datasets/valid;
!mkdir /content/datasets/train/images;mkdir /content/datasets/train/labels;
!mkdir /content/datasets/valid/images;mkdir /content/datasets/valid/labels;

### 데이터 스플릿 & 파일 이동
1. 재현을 위한 난수 고정 : 2024
2. 테스트셋 데이터 20%
- 위 설정에 맞게 데이터를 나누고, 위 과정에서 생성한 폴더에 이동시키세요.

# **Load ex_train**

In [ ]:
img_path = '/content/images/'
lab_path = '/content/labels/obj_train_data/'

In [ ]:
ex_img_path = '/content/_train/train/images/'
ex_img_list = os.listdir(ex_img_path)
ex_lab_path = '/content/_train/train/labels/'
ex_lab_list = os.listdir(ex_lab_path)

for ex_tr_img, ex_tr_lab in zip(ex_img_list,ex_lab_list) :
    shutil.move(ex_img_path + ex_tr_img, img_path)
    shutil.move(ex_lab_path + ex_tr_lab, lab_path)

# Load Img

In [ ]:
# Dataset metadata 입력
img_path = '/content/images/'
lab_path = '/content/labels/obj_train_data/'

img_list = os.listdir(img_path)
img_list = sorted(img_list)
lab_list = os.listdir(lab_path)
lab_list = sorted(lab_list)


# 파일 담아오기
data_files = []
for img, lab in zip(img_list,lab_list):
    data_files.append([glob.glob(img_path+img), glob.glob(lab_path+lab)])

# img size (640,640)

In [ ]:
# Pillow 라이브러리 불러오기
from PIL import Image

for i in tqdm(range(1,201)):
  img = Image.open(f'/content/images/roof ({i}).jpg')
  img = img.resize((640,640))
  img.save(f'/content/images/roof ({i}).jpg')


#test_img

test_path = "/content/o_test/test/images/"

for test_file in tqdm(os.listdir(test_path)):
  img = Image.open(test_path + test_file)
  img = img.resize((640,640))
  img.save(test_path + test_file)


# HSV

In [ ]:
#HSV
import cv2
for i in tqdm(range(1,len(img_list)+1)):
  rgbimg = cv2.imread(f'/content/images/roof ({i}).jpg')
  hsvimg = cv2.cvtColor(rgbimg, cv2.COLOR_RGB2HSV)
  cv2.imwrite(f'/content/images/roof ({i}).jpg', hsvimg)



for test_file in tqdm(os.listdir(test_path)):
  rgbimg = cv2.imread(test_path + test_file)
  hsvimg = cv2.cvtColor(rgbimg, cv2.COLOR_RGB2HSV)
  cv2.imwrite(test_path + test_file, hsvimg)


# Gray scale

In [ ]:
#GRAY
import cv2
for i in tqdm(range(1,len(img_list)+1)):
  rgbimg = cv2.imread(f'/content/images/roof ({i}).jpg')
  grayimg = cv2.cvtColor(rgbimg, cv2.COLOR_RGB2GRAY)
  cv2.imwrite(f'/content/images/roof ({i}).jpg', grayimg)

for test_file in tqdm(os.listdir(test_path)):
  rgbimg = cv2.imread(test_path + test_file)
  grayimg = cv2.cvtColor(rgbimg, cv2.COLOR_RGB2GRAY)
  cv2.imwrite(test_path + test_file, grayimg)

# Sharping Filter

In [ ]:
def sharp(img):
    kernel_sharp = np.array((
        [-2, -2, -2],
        [-2, 17, -2],
        [-2, -2, -2]), dtype='int')
    return cv2.filter2D(img, -1, kernel_sharp)

In [ ]:
for i in tqdm(range(1,len(img_list)+1)):
  img = cv2.imread(f'/content/images/roof ({i}).jpg')
  sh_img = sharp(img)
  cv2.imwrite(f'/content/images/roof ({i}).jpg', sh_img)

for test_file in tqdm(os.listdir(test_path)):
  img = cv2.imread(test_path + test_file)
  sh_img = sharp(img)
  cv2.imwrite(test_path + test_file, sh_img)

# data split

In [ ]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(data_files, test_size=.2, random_state=2024)

In [ ]:
train_path = '/content/datasets/train/'
valid_path = '/content/datasets/valid/'

for tr_img, tr_lab in train:
    shutil.move(tr_img[0], train_path +'images')
    shutil.move(tr_lab[0], train_path +'labels')

for va_img, va_lab in val:
    shutil.move(va_img[0], valid_path + 'images')
    shutil.move(va_lab[0], valid_path + 'labels')

### YOLO 모델에 적용할 YAML 생성하기
- 지붕에는 두 가지 형태가 있습니다. 클래스 구분에 주의하세요.
- cool roof
- generic roof

In [ ]:
import yaml

data = {
    'train': '../train/images',
    'val': '../valid/images',
    'nc': 2,
    'names': ['cool roof', 'generic roof']
}

with open('/content/datasets/data.yaml', 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)


### YOLO v8 모델
- yaml 파일의 경로 설정에 주의하세요.

In [ ]:
!pip install ultralytics==8.0.196


In [ ]:
from ultralytics import YOLO, settings

In [ ]:
model = YOLO(model='yolov8n.pt', task='detect')

In [ ]:
settings['datasets_dir'] = '/content/'
settings.update()

In [ ]:
model.train(data='/content/datasets/data.yaml',
            epochs=150,
            patience=30,
            pretrained=False,
            verbose=True,
            seed=2024,
            hsv_s=0.8,
            )

# **Model.val**

In [ ]:
model.val()
# HSV # 640x640 img

In [ ]:
model.val()
# 640x640 img

In [ ]:
model.val()
# Gray scale # 640x640 img

In [ ]:
model.val()
# add 171 train data #HSV

In [ ]:
model.val()
# add 171 train data #Grayscale

In [ ]:
model.val()
# add 171 train data #sharping

In [ ]:

pred = model.predict(test_path,conf=.6,save=True, iou=0.2 )

In [ ]:

pred_path ='/content/runs/detect/predict/'

for test_file in tqdm(os.listdir(pred_path)):
  hsvimg = cv2.imread(pred_path + test_file)
  rgbimg = cv2.cvtColor(hsvimg, cv2.COLOR_HSV2RGB)
  cv2.imwrite('/content/runs/detect/predict/' + test_file, rgbimg)


# Sharping filter를 적용한 이미지의 예측 성승이 우수함.
- generic roof를 판단하는데 건물의 외각선 추출이 성는의 영향을 미침

train data 1085개에 대해서 학습을 진행하고자 하였으나, 시간이 부족하여 성능 비교를 마치지 못하였음.